In [41]:

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import numpy as np

from tqdm import trange
from  torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [33]:
# Hyper-Parameters
epoch = 100
batch_size = 16
learning_rate = 0.001

In [9]:
# Download_dataset
mnist_train = dset.MNIST('./mnist_data/', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST('./mnist_data/', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

Processing...
Done!


In [26]:
print("train set : %d" % mnist_train.__len__())
print("test set : %d" % mnist_test.__len__())

img1, label1 = mnist_train.__getitem__(0)
img2, label2 = mnist_test.__getitem__(0)

print(img1.size(), label1)
print(img2.size(), label2)

train set : 60000
test set : 10000
torch.Size([1, 28, 28]) 5
torch.Size([1, 28, 28]) 7


In [27]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True)

### CNN model

In [28]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 16, 5), # 16 ch * ((28 - 5_kernel / 1_stride) + 1)
                        nn.ReLU(),
                        nn.Conv2d(16, 32, 5), # 32 ch * ((24-5) +1) * 20
                        nn.ReLU(),
                        nn.MaxPool2d(2, 2),  # 32 ch * (((20-2)/2)+1) * 10
        )
        
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, 5), # 64 ch * ((10-5)+1) * 6
                        nn.ReLU(),
                        nn.Conv2d(64, 128, 5), # 128 ch * ((6-5) + 1) * 2
                        nn.ReLU(),
        )
        self.fc = nn.Linear(in_features=128*2*2, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size, -1)
        out = self.fc(out)
        return out

In [29]:
cnn = CNN()

In [30]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=learning_rate)

### Train phase

In [42]:
for e in trange(epoch):
    if e+1 % 100 == 0:
        print(loss)
    for j, [image, label] in enumerate(train_loader):
        image = Variable(image).cuda()
        label = Variable(label).cuda()
        cnn.cuda()
        
        optimizer.zero_grad()
        result = cnn.forward(image)
        loss = loss_function(result, label)
        loss.backward()
        optimizer.step()

  0%|                                                       | 0/100 [00:00<?, ?it/s]

Variable containing:
1.00000e-05 *
  2.2411
[torch.cuda.FloatTensor of size 1 (GPU 0)]



100%|█████████████████████████████████████████████| 100/100 [31:34<00:00, 18.95s/it]


### Test Phase

In [44]:
correct = 0
total = 0

for image, label in test_loader:
    image = Variable(image).cuda()
    
    result = cnn.forward(image).cuda()
    _, predicted = torch.max(result.data, 1)
    total += label.size(0)
    correct += (predicted == label.cuda()).sum()

print('Test Accuracy of the model on 10000 test images : %f %%' % (100 * correct / total))

Test Accuracy of the model on 10000 test images : 98.740000 %


### Result 

약 500회 training 실행결과 : 98.74%
